# 1. 모델 기본 정보

In [47]:
# 참고 git 허브 주소
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'
repo_model = 'https://github.com/tensorflow/models.git'
repo_blog_url = 'https://towardsdatascience.com/custom-object-detection-using-tensorflow-from-scratch-e61da2e10087'

# 학습 횟수 (많을 수록 정확도가 높다)
num_steps = 3000

# 검토 횟수
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2' : {
        'model_name' : 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file' : 'ssd_mobilenet_v2_coco.config',
        'batch_size' : 12
    },
    'faster_rcnn_inception_v2' : {
        'model_name' : 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file' : 'faster_rcnn_inception_v2_pets.config',
        'batch_size' : 8
    },
    'rfcn_resnet101' : {
        'model_name' : 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file' : 'rfcn_resnet101_pets.config',
        'batch_size' : 8
    }
}

# 모델 이름
selected_model = 'ssd_mobilenet_v2'


# 사용할 모델
MODEL = MODELS_CONFIG[selected_model]['model_name']

pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# batch 크기
batch_size = MODELS_CONFIG[selected_model]['batch_size']



# 2. 시스템 환경 세팅

In [1]:
#  /root/content/ 위치로 이동
%cd /content
%pwd

# 깃 클론
!git clone --quiet https://github.com/tensorflow/models.git

# packages, protos 설치
!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python -tk

! pip install -q Cython contextlib2 pillow lxmlmatplotlib

!pip install -q pycocotools

!pip install lvis

!pip install tensorflow_io

!pip install official

# protos 세팅
%cd /content/models/research
%pwd
!protoc ./object_detection/protos/*.proto --python_out=.

# slim 환경변수 설정
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install numpy
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools






/content
E: The value 'k' is invalid for APT::Default-Release as such a release is not available in the sources
ERROR: Could not find a version that satisfies the requirement lxmlmatplotlib (from versions: none)
ERROR: No matching distribution found for lxmlmatplotlib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
/content/models/research
Found existing installation: pycocotools 2.0.6
Uninstalling pycocotools-2.0.6:
  Successfully uninstalled pycocotools-2.0.6
  Using cached pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
python3: can't open file '/content/models/research/object_detection/builers/model_builder_test.py': [Errno 2] No such file or directory


# 3. model builder test 실행

In [8]:
!python ./object_detection/builders/model_builder_test.py

2023-07-14 07:41:16.059766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 07:41:16.896698: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [30]:
%cd /content/models
%rm -r data

/content/models


In [25]:
%pwd
%cd /content/drive/MyDrive/
%mv ./BCCD.zip /content/models

/content/drive/MyDrive


In [32]:
%cd /content/models/
%mkdir data
%mv ./BCCD.zip ./data/
%cd data
!unzip ./BCCD.zip

/content/models
mkdir: cannot create directory ‘data’: File exists
/content/models/data
Archive:  ./BCCD.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/cells.tfrecord     
 extracting: test/cells_label_map.pbtxt  
   creating: train/
 extracting: train/cells.tfrecord    
 extracting: train/cells_label_map.pbtxt  
   creating: valid/
 extracting: valid/cells.tfrecord    
 extracting: valid/cells_label_map.pbtxt  


# 4.tfrecord 버전

In [52]:
test_record_fname = '/content/models/data/test/cells.tfrecord'
train_record_fname = '/content/models/data/train/cells.tfrecord'
label_map_pbtxt_fname = '/content/models/data/train/cells_label_map.pbtxt'

In [44]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [45]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

In [48]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [49]:
import os
import sys
sys.path.append('/content/models/research/')

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [53]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:

    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [54]:
!cat {pipeline_fname}

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

In [55]:
model_dir = 'training/'

!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok = True)

In [59]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

ValueError: ignored

In [60]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

2023-07-14 09:39:42.452934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
Use tf.keras instead.
W0714 09:39:44.848074 137259432486720 deprecation.py:364] From /content/models/research/object_detection/model_main.py:63: RunConfig.__init__ (from tensorflow_estimator.python.estimator.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
W0714 09:39:44.850995 137259432486720 model_lib.py:838] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 3000
I0714 09:39:44.851128 137259432486720 config_util.py:552] Maybe overwriting train_steps: 3000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0714 09:39:44.851189 137259432486720 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0714 09:39:44.851239 137259432486720 config_util.

In [ ]:
import os
output_directory = './fine_tuned_model'
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
from google.colab import files
files.download(pb_fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>